In [ ]:
import os
with open('tales.txt', 'r', encoding='utf-8') as file:
    data = file.read()
lines = data.split('\n')
split_ratio = 0.8
split_index = int(len(lines) * split_ratio)
train_lines = lines[:split_index]
test_lines = lines[split_index:]
output_dir = 'data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(os.path.join(output_dir, 'train.txt'), 'w', encoding='utf-8') as train_file:
    train_file.write('\n'.join(train_lines))
with open(os.path.join(output_dir, 'test.txt'), 'w', encoding='utf-8') as test_file:
    test_file.write('\n'.join(test_lines))

print('Data successfully split and saved into train.txt and test.txt')


Data successfully split and saved into train.txt and test.txt


In [ ]:
!pip install transformers
!pip install tf-keras

In [ ]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

In [ ]:
train_path = '/content/data/train.txt'
test_path = '/content/data/test.txt'

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
print('vocabulary size: %d, max squence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))
print('tokenize sequence "Once upon a time in a little village":', tokenizer('Once upon a time in a little village'))

vocabulary size: 50257, max squence length: 1024
tokenize sequence "Once upon a time in a little village": {'input_ids': [7454, 2402, 257, 640, 287, 257, 1310, 7404], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=128)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
print(tokenizer.decode(train_dataset[2]))

 Wellington, south of Arizona and east of where you are sitting right now. It was a dark night, with no birds nor chickens, and it was raining a silent rain. There were too many stars to count, and not enough clouds to cover them. I like chickens. But anyway, what was I doing in this smelly house?
"I was laughing at the boxes of dog food with you, remember!?" said Stevens grandma, but she was wrong. What was I doing? That's right, I was writing this story. Now back to the story.
I needed to find something, so looked for it, and the


In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
!pip install accelerate -U

In [ ]:
training_args = TrainingArguments(
    output_dir = 'data/out',
    overwrite_output_dir = True,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    learning_rate = 5e-5,
    num_train_epochs = 3,
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=3.3911622365315757, metrics={'train_runtime': 62.388, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.048, 'total_flos': 979845120000.0, 'train_loss': 3.3911622365315757, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [ ]:
generator = pipeline('text-generation', tokenizer='gpt2', model='data/out')

In [ ]:
print(generator('sum of 2+3=?', max_length=40)[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sum of 2+3=?

If you are still not sure what to do, get a hold of it. Just use the one that you feel most comfortable with: the one with the


In [ ]:
text_beam = generator('Once upon a time',
                      max_length=40,
                      num_beams=5)
print(text_beam[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, it seemed like a matter of years, but now it seemed like a matter of years, and now it seemed like a matter of years.

It was as if the


In [ ]:
text_random_sampling = generator('How are you?',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.7)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How are you? Is it ok to be different?

"I know what you're thinking." His voice was suddenly quieter, beginning to sound like he'd been breathing heavily. He turned around


In [ ]:
text_k_sampling = generator('who are you?',
                            max_length=40,
                            top_k=40,
                            do_sample=True)
print(text_k_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


who are you? Is this a movie, or is there something more to the situation there?


What I'm missing in this movie is the action. The tension. The violence. The tension


In [ ]:
text_p_sampling = generator('HI there',
                            max_length=40,
                            top_k=0,
                            top_p=0.92,
                            do_sample=True)
print(text_p_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


HI there. I'm just a subscriber now and the money is out of reach. But I still have the ability to write. This is cool. Hopefully the reputation will change too.
